In [7]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langgraph.graph import StateGraph, START
from typing_extensions import TypedDict, List
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from supabase.client import create_client

## Step 1: Create a function for Deepseek Call

In [8]:
import requests
import os
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
DEEPSEEK_URL = "https://api.deepseek.com/v1/chat/completions"

def call_deepseek(question, context):
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Answer the question based on the context below:\n\nContext:\n{context}\n\nQuestion:\n{question}"}
    ]
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 512
    }

    response = requests.post(DEEPSEEK_URL, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]


In [11]:
url="https://www.healthline.com/health/bodyweight-workout"

loader = WebBaseLoader([url])
docs = loader.load()
splitter = RecursiveCharacterTextSplitter()
splits = splitter.split_documents(docs)
 
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
vectorstore_in_memory = FAISS.from_documents(splits, embeddings)

In [14]:
query = "upper body program"
top_k = 4
retrieved_docs = vectorstore_in_memory.similarity_search(query, k=top_k)

# Combine context
context = "\n\n".join(doc.page_content for doc in retrieved_docs)

# Get answer from DeepSeek
answer = call_deepseek(query, context)
print("\n DeepSeek Answer:\n", answer)


 DeepSeek Answer:
 Here’s an **upper body-focused bodyweight workout program** designed to build strength and muscle. This routine is suitable for **intermediate to advanced** fitness levels but can be modified for beginners.

---

### **Upper Body Bodyweight Circuit**  
**Instructions:**  
- Perform each exercise for **45–60 seconds** (or 8–15 reps per side where applicable).  
- Rest **30–60 seconds** between exercises.  
- Complete **2–3 rounds** of the circuit.  

#### **1. Pushup Variations**  
   - *Standard Pushups*: Keep core tight, lower chest to floor.  
   - *Wide-Grip Pushups*: Hands wider than shoulders to target chest.  
   - *Diamond Pushups*: Hands close (thumbs/index fingers touching) for triceps.  

#### **2. Pike Pushups** (Shoulders)  
   - Start in a downward dog position (hips high), bend elbows to lower head toward floor, then push back up.  

#### **3. Triceps Dips**  
   - Use a bench or chair: Hands on edge, legs extended, lower hips by bending elbows to 90 d